In [1]:
import datetime
import pandas as pd

In [2]:
def na_table(x):
    na = pd.DataFrame(x.isna().sum())
    nna = pd.DataFrame(x.notna().sum())
    na.reset_index(inplace=True)
    nna.reset_index(inplace=True)
    na = na[na[0]>0]
    na = na.merge(nna, on='index')
    na.columns = ['col', 'isNA', 'notNA']
    return na

In [3]:
data_dir = '../train/'
features_dir = '../features/'
tax_info = pd.read_csv(data_dir + 'tax_info.csv',encoding = 'gbk')
tax_info.drop_duplicates(inplace=True)
t = tax_info.copy()
na = na_table(tax_info)
na

,col,isNA,notNA
0,TAXATION_BASIS,3264,21350
1,TAX_RATE,3264,21350
2,DEDUCTION,3490,21124


In [5]:
tax_info.START_DATE = pd.to_datetime(tax_info.START_DATE)
tax_info.END_DATE = pd.to_datetime(tax_info.END_DATE)
tax_info['gap'] = tax_info.END_DATE-tax_info.START_DATE
tax_info.gap = tax_info.gap.apply(lambda x:x.days + 1)
temp = pd.DataFrame(tax_info.groupby('id').sum())
tax_info = pd.DataFrame(tax_info.groupby('id').count())
tax_info = pd.DataFrame([tax_info.TAX_CATEGORIES, tax_info.TAX_ITEMS]).T
tax_info = tax_info.merge(temp, on='id')
tax_info.TAX_RATE = tax_info.TAX_RATE / tax_info.TAX_CATEGORIES
# tax_info = tax_info[(tax_info.TAXATION_BASIS == 0) & (tax_info.TAX_RATE == 0) & (tax_info.DEDUCTION == 0) & (tax_info.TAX_AMOUNT != 0)].replace(0, pd.NA)
tax_info.to_csv(features_dir + 'tax_info.csv')
tax_info.head()

,TAX_CATEGORIES,TAX_ITEMS,TAXATION_BASIS,TAX_RATE,DEDUCTION,TAX_AMOUNT,gap
id,,,,,,,
216bd2aaf4d079243f3c0bd3d6d28333c790bd3aee0ddad8,45,45,850992.56,0.032127,0.57,1714.89,1350
216bd2aaf4d079248a1cb9c41425810a25d29c1fc1d1c15a,4,4,0.00,0.000000,0.00,25038.50,124
216bd2aaf4d07924caa4f30fb76969cba69358e90e310f5e,1,1,0.00,0.000000,0.00,2400.00,31
216bd2aaf4d07924d90a8951ef6a87c01e2f2ec78aa14698,2,2,0.00,0.000000,0.00,204.60,62
47645761dc56bb8c0ee994e2cf201d523319b7160f6722f8,4,4,0.00,0.000000,0.00,73.94,124


In [6]:
pd.DataFrame(tax_info[(tax_info.TAXATION_BASIS == 0) & (tax_info.TAX_RATE == 0) & (tax_info.DEDUCTION == 0) & (tax_info.TAX_AMOUNT != 0)].replace(0, pd.NA).isna().sum())

,0
TAX_CATEGORIES,0
TAX_ITEMS,0
TAXATION_BASIS,421
TAX_RATE,421
DEDUCTION,421
TAX_AMOUNT,0
gap,0


In [7]:
tax_info = t
tax_info['year'] = tax_info.START_DATE.apply(lambda x:x.split('/')[0])
tax_info.fillna(-1, inplace=True)
tax_info = tax_info.pivot_table(index='id', columns=['TAX_CATEGORIES', 'year']).fillna(0)
tax_info.replace(-1, pd.NA, inplace=True)
tax_info.reset_index(inplace=True)
cols = ['id']
cols.extend(['tax' + str(i) for i in range(len(tax_info.columns)-1)])
tax_info.columns = cols
tax_info.head()

,id,tax0,tax1,tax2,tax3,tax4,tax5,tax6,tax7,tax8,...,tax246,tax247,tax248,tax249,tax250,tax251,tax252,tax253,tax254,tax255
0,216bd2aaf4d079243f3c0bd3d6d28333c790bd3aee0ddad8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,216bd2aaf4d079248a1cb9c41425810a25d29c1fc1d1c15a,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,216bd2aaf4d07924caa4f30fb76969cba69358e90e310f5e,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,216bd2aaf4d07924d90a8951ef6a87c01e2f2ec78aa14698,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,47645761dc56bb8c0ee994e2cf201d523319b7160f6722f8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
tax_info.set_index(keys='id', inplace=True)
tax_info.to_csv(features_dir + 'tax_info_.csv')